In [1]:
%pip install SpeechRecognition
%pip install gtts
%pip install pyaudio
%pip install pandas
%pip install re
%pip install tensorflow
%pip install numpy
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import speech_recognition as sr
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
# Verinin dosyadan okunması, parçalanması ve eğitim için hazırlanması
data = pd.read_csv("_data.csv", header=0, delimiter="\t", quoting=3)
data = data[['Review', 'Sentiment']]

turkish_chars = "a|b|c|ç|d|e|f|g|ğ|h|ı|i|j|k|l|m|n|o|ö|p|r|s|ş|t|u|ü|v|y|z|0-9"

data['Review'] = data['Review'].astype(str)
data['Review'] = data['Review'].apply(lambda x: x.lower())
data['Review'] = data['Review'].apply((lambda x: re.sub('[^'+turkish_chars+'\s]','',x)))

# Metin verisini sayısal verilere dönüştürme
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(data['Review'].values)
X = tokenizer.texts_to_sequences(data['Review'].values)
X = pad_sequences(X, maxlen=400)
Y = pd.get_dummies(data['Sentiment']).values

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
# Model oluşturma
model = Sequential()
model.add(Embedding(25000, 128, input_length=X.shape[1]))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modelin başlangıç ağırlıklarını sabitleme
tf.random.set_seed(42)

# Modelin eğitimi
history = model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=(X_test, Y_test), verbose=1)


In [ ]:
# Modelin değerlendirilmesi
score = model.evaluate(X_test, Y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Modelin kaydedilmesi
model.save("sentiment_analysis_model.h5")

In [ ]:
# Ses kaydından metin çevirme ve duygu analizi
def perform_sentiment_analysis(audio_file_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio_data = r.listen(source)
        try:
            text = r.recognize_google(audio_data, language="tr-TR")
            print("Ses Metni:")
            print(text)
            sequences = tokenizer.texts_to_sequences([text])
            data = pad_sequences(sequences, maxlen=400)
            prediction = model.predict(data)
            print("Duygu Analizi Sonucu:")
            print("Olumsuz Olasılık:", prediction[0][0])
            print("Olumlu Olasılık:", prediction[0][1])
            return prediction
        except sr.UnknownValueError:
            print("Ses anlaşılamadı.")
            return None
        except sr.RequestError as e:
            print("Hata:", e)
            return None

In [ ]:
# Duygu analizini gerçekleştirmek için bir örnek
perform_sentiment_analysis("res.wav")

# Eğitim sürecinin görselleştirilmesi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
